In [1]:
import ijson
import pandas as pd
import numpy as np
import re
import string
from langdetect import detect
import nltk
from stop_words import get_stop_words
from nltk.corpus import stopwords
import tqdm
from collections import Counter
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

In [2]:
Review=pd.read_csv('C:/Users/Fannie/Desktop/yelpreview.csv')

In [7]:
len(Review)

8635403

In [5]:
Review_dropna=Review
Review_dropna=Review_dropna.dropna()

In [6]:
len(Review_dropna)

8635401

In [19]:
# ueful
Review_29up=Review_dropna[(Review_dropna['useful']>= 29)]
len(Review_29up)

13637

In [20]:
# moderate useful
Review_10to20 = Review_dropna[(Review_dropna['useful']>10)&(Review_dropna['useful']<=20)]
len(Review_10to20)

91933

In [21]:
# not useful
Review_0to1 = Review_dropna[(Review_dropna['useful']>=0)&(Review_dropna['useful']<=1)]
len(Review_0to1)

6585177

In [70]:
Review_useful=Review_29up
Review_moderate= Review_10to20.sample(n=17358)
Review_notuseful=Review_0to1.sample(n=24256)
Review_project = pd.concat([Review_useful,Review_moderate, Review_notuseful], axis=0)

In [71]:
len(Review_project)

55251

In [72]:
Review_project['useful_level']=""

In [73]:
Review_project.loc[ (Review_project['useful']>10) | (Review_project['useful']<=20) , 'useful_level' ] = "moderate useful"
Review_project.loc[Review_project['useful']>=29,'useful_level']="useful"
Review_project.loc[Review_project['useful']<=1,'useful_level']="not useful"

In [74]:
Review_project['useful_level'].value_counts()

not useful         24256
moderate useful    17358
useful             13637
Name: useful_level, dtype: int64

In [75]:
from sklearn.utils import shuffle
Review_project = shuffle(Review_project)

In [76]:
language = []
for i in list(range(len(Review_project))):   
    m = detect(Review_project.iat[i,8])
    language.append(m)

In [77]:
Review_project['language']=language
Review_project = Review_project[Review_project['language'] == 'en']# keep english

In [78]:
len(Review_project)

55187

In [79]:
Review_project.to_csv('Review_project.csv', sep=',', index=False)

data processing

In [3]:
Review_project=pd.read_csv('Review_project.csv')

In [4]:
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and   characters and then strip)
    text = re.sub(r'[^\w\s]', '',str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

In [5]:
lst_stopwords = nltk.corpus.stopwords.words("english")

In [6]:
Review_project["text_clean"] = Review_project["text"].apply(lambda x: 
          utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, 
          lst_stopwords=lst_stopwords))
Review_project.at[8,'text_clean']

'blackened redfish taco celis white restaurant never let delicious fresh fish cooked well taco come onion cilantro lettuce white sauce healthy slice lime sloppy fantastic'

Readability

In [7]:
import textstat
#textstat.flesch_reading_ease(Review_project.iat[7,8])
reading_ease= []
for i in list(range(len(Review_project))):   
    ease = textstat.flesch_reading_ease(Review_project.at[i,'text'])
    reading_ease.append(ease)

In [8]:
Review_project['read_ease']=""
Review_project['read_ease']=reading_ease

In [16]:
Review_project.to_csv('Review_project_readease.csv', sep=',', index=False)

polarity and subjectivity

In [15]:
Review_project=pd.read_csv('Review_project_readease.csv')

In [16]:
from textblob import TextBlob

In [17]:
def detect_polarity(text):
    return TextBlob(text).sentiment.polarity

In [18]:
def detect_subjectivity(text):
    return TextBlob(text).sentiment.subjectivity

In [19]:
Review_project['polarity'] = Review_project.text.apply(detect_polarity)

In [20]:
Review_project['subjectivity'] = Review_project.text.apply(detect_subjectivity)

In [21]:
Review_project.head()

,Unnamed: 0,review_id,user_id,business_id,stars,useful,funny,cool,text,date,useful_level,language,text_clean,read_ease,polarity,subjectivity
0,1073206,JdReKgETiiJEDmshrO4TLw,pyarmAnR-i-qookQamqRTA,V2GOReqPvr8qpCC7sWfoTw,1.0,17,1,0,Just to let this car company that people DO re...,2014-03-06 12:38:52,moderate useful,en,let car company people read yelp check review ...,85.49,-0.500000,1.000000
1,6229216,zL4se_Ixdcl8kvTOHCS3rg,s16-BUo-orUsELvMu5ocKg,VH0Ib9S3E-dxbQdQC4rffg,1.0,15,6,4,Mistral was the worst dining experience I have...,2010-07-22 18:08:01,moderate useful,en,mistral worst dining experience ever life bad ...,79.19,0.029864,0.502499
2,7607157,wlLQ42QMHiQMJISCTUVX5A,aQA2eIdEC3_ZDHwUregu8A,wygOtij7aEb7dTdTFFnOcw,5.0,0,0,0,Took a VIP tour with DeMon of the Atlanta Merc...,2020-02-04 16:05:20,not useful,en,took vip tour demon atlanta mercedes benz stad...,-32.40,0.422500,0.696111
3,141509,OMw52-Eo-BYFHdUldLMc5Q,mmr-P0i__uQJHGXXeVjXxg,0Ng-wDNyA-96uaJZP8EKZg,4.0,40,0,0,When I first came here for the first time last...,2015-03-09 22:42:06,useful,en,first came first time last year best friend ap...,83.96,0.168452,0.419841
4,7962637,RWq78T9FIyauVHaHj7yl9g,CyCI71vLDa923zOMeDDA5w,WUfZXjTtBUbezJE7LnWABw,3.0,0,0,0,"I only got the take-out, so I cannot speak of ...",2013-03-22 20:47:40,not useful,en,got takeout cannot speak service food good wou...,51.86,0.320000,0.780000


In [22]:
Review_project.to_csv('Review_project_polarity+readease+sub.csv', sep=',', index=False)

In [29]:
corpus=Review_project.at[5,'text_clean']
text1=nltk.Text(Review_project.at[5,'text_clean'])
tags=nltk.pos_tag(text1)
from collections import Counter
counts = Counter(tag for text1,tag in tags)
counts

Counter({'NN': 540,
         'VBP': 53,
         'JJ': 92,
         'VBD': 24,
         'IN': 7,
         'DT': 55,
         'NNP': 117,
         'VBZ': 42,
         'FW': 3,
         'CD': 10,
         'NNS': 1,
         '$': 1,
         'VB': 3,
         'CC': 2})

ratio of adjectives

In [33]:
sentence = nltk.word_tokenize(Review_project.at[5,'text_clean'])
sent = nltk.pos_tag(sentence)
listnn = ([s for s in sent if s[1] == 'JJ'])
adj_ratio=len(listnn)/len(sentence)


0.13333333333333333


In [34]:
Adj_ratio= []
for i in list(range(len(Review_project))):   
    sentence = nltk.word_tokenize(Review_project.at[i,'text_clean'])
    tags = nltk.pos_tag(sentence)
    adj = ([s for s in tags if s[1] == 'JJ'])
    adj_ratio=len(adj)/len(sentence)
    Adj_ratio.append(adj_ratio)

In [35]:
Review_project['Adj_ratio']=Adj_ratio

In [36]:
Review_project.to_csv('Review_project_sentiment.csv', sep=',', index=False)